In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor


/Volumes/Data/AI Projects/parallelization-multi-document-summarizer/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
# LLM Call
def call_llm(prompt):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [4]:
# Summarize one document
def summarize_document(doc_text):
    prompt = f"Summarize the following document in 5 bullet points:\n\n{doc_text}"
    return call_llm(prompt)

# Summarize all documents in parallel
def summarize_documents_parallel(doc_texts):
    with ThreadPoolExecutor(max_workers=5) as executor:
        summaries = list(executor.map(summarize_document, doc_texts))
    return summaries

# Merge all summaries into one overview
def merge_summaries(summaries):
    combined_text = "\n\n".join(f"Summary {i+1}:\n{s}" for i, s in enumerate(summaries))
    prompt = f"You are a helpful analyst. Combine the following summaries into one coherent paragraph:\n\n{combined_text}"
    return call_llm(prompt)

# Gradio Interface Handler
def process_files(files):
    doc_texts = []
    for file in files:
        if file.name.endswith(".txt"):
            with open(file.name, "r", encoding="utf-8") as f:
                doc_texts.append(f.read())


    if not doc_texts:
        return "No valid .txt files uploaded.", None

    summaries = summarize_documents_parallel(doc_texts)
    merged_summary = merge_summaries(summaries)

    summary_output = "\n\n".join(
        f"📄 Document {i+1} Summary:\n{summary}" for i, summary in enumerate(summaries)
    )

    return summary_output, f"🧠 Merged Summary:\n\n{merged_summary}"

In [7]:
with gr.Blocks() as demo:
    gr.Markdown("# 📚 Multi-Document Summarizer (Agentic AI)")
    gr.Markdown("Upload multiple .txt documents and get summaries & a merged overview.")

    with gr.Row():
        file_input = gr.File(label="Upload .txt files", file_types=[".txt"], file_count="multiple")
        submit_btn = gr.Button("Summarize")

        summary_output = gr.Textbox(label="Individual Summaries", lines=20)
        merged_output = gr.Textbox(label="Merged Overview", lines=10)

    submit_btn.click(fn=process_files, inputs=[file_input], outputs=[summary_output, merged_output])

demo.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
